<a href="https://colab.research.google.com/github/Sreyas2403/GenAI-Research-Agent-/blob/main/ResearchAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai wikipedia langgraph


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.2 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=59cea2814d2e0d05cdf708f75b1624990898b4db6ae86759a55bc81512b78fb2
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.18
    Uninstalling langchain-0.3.18:
      Successfully uninstalled langchain-0.3.18


In [5]:
import langgraph
from langchain.tools import Tool
from langchain.utilities import WikipediaAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langgraph.graph import END, StateGraph
from pydantic import BaseModel

# Define the input and output schema
class AgentState(BaseModel):
    input: str
    output: str = ""

# Define the Wikipedia search tool
wiki_tool = Tool(
    name="Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Search Wikipedia for information on a given topic."
)

# Define the agent function
def agent_function(state: AgentState):
    user_input = state.input

    # Call Wikipedia search tool
    search_results = wiki_tool.run(user_input)

    return AgentState(input=user_input, output=search_results)

# Build the LangGraph
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent_function)
workflow.set_entry_point("agent")
workflow.add_edge("agent", END)

# Compile the graph
app = workflow.compile()

# Run the agent in an interactive loop
while True:
    user_query = input("Ask me any topic for your research: ")
    if user_query.lower() in ["exit", "quit"]:
        print("See you Again!")
        break
    result = app.invoke(AgentState(input=user_query))
    print("\nResponse:\n", result["output"], "\n")


Ask me any topic for your research: Olympics 2024

Response:
 Page: 2024 Summer Olympics
Summary: The 2024 Summer Olympics (French: Les Jeux Olympiques d'été de 2024), officially the Games of the XXXIII Olympiad (French: Jeux de la XXXIIIe olympiade de l'ère moderne) and branded as Paris 2024, were an international multi-sport event held from 26 July to 11 August 2024 in France, with several events started from 24 July. Paris was the host city, with events (mainly football) held in 16 additional cities spread across metropolitan France, including the sailing centre in the second-largest city of France, Marseille, on the Mediterranean Sea, as well as one subsite for surfing in Tahiti, French Polynesia.
Paris was awarded the Games at the 131st IOC Session in Lima, Peru, on 13 September 2017. After multiple withdrawals that left only Paris and Los Angeles in contention, the International Olympic Committee (IOC) approved a process to concurrently award the 2024 and 2028 Summer Olympics to 

In [6]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=11c00664cc98f22d3a74c84eb46f2fc63201803ab4188395d1f62cd27d17beea
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [11]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# Sample reference (ground truth) - Manually retrieved Wikipedia summary
reference_text = "Albert Einstein was a theoretical physicist, widely known for developing the theory of relativity."

# Function to calculate BLEU & ROUGE scores
def evaluate_response(generated_text, reference_text):
    # BLEU Score Calculation
    reference_tokens = [reference_text.split()]
    generated_tokens = generated_text.split()
    bleu = sentence_bleu(reference_tokens, generated_tokens)

    # ROUGE Score Calculation
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores = scorer.score(reference_text, generated_text)

    print(f"\n BLEU Score: {bleu:.4f}")
    print(f" ROUGE-1: {rouge_scores['rouge1'].fmeasure:.4f}")
    print(f" ROUGE-2: {rouge_scores['rouge2'].fmeasure:.4f}")
    print(f" ROUGE-L: {rouge_scores['rougeL'].fmeasure:.4f}\n")

# Example usage after getting agent output
generated_text = "Albert Einstein was a physicist known for his theory of relativity."
evaluate_response(generated_text, reference_text)



 BLEU Score: 0.2879
 ROUGE-1: 0.8000
 ROUGE-2: 0.5217
 ROUGE-L: 0.8000

